In [ ]:
import pandas as pd
import re
ndf=pd.read_csv('ndf.csv')
# Find all words ending with '.jpg' in each row of text and associate them with the same value
pic = []
shortcode = []
for i,rows in ndf.iterrows():
    row_matches = re.findall(r'\b\w+\.jpg\b', ndf['pic'][i])
    pic.extend(row_matches)
    shortcode.extend([ndf['shortcode'][i]] * len(row_matches))

# Create a DataFrame with the matches and associated values
imgdf = pd.DataFrame({'pic': pic, 'value': shortcode})

imgdf.to_csv('imgdf.csv')

In [5]:
import pandas as pd
import re
imgdf=pd.read_csv('/workspace/persistent/imgdf.csv')

In [6]:
import pandas as pd

def add_prefix(row):
    prefix = '/workspace/persistent/img_resized/'
    return prefix + row

# Assuming you have a DataFrame named 'df'
imgdf['pic'] = imgdf['pic'].apply(add_prefix)

In [9]:
imgdf

,Unnamed: 0,pic,shortcode,image_hidden,image_attention,image_representation
0,0,/workspace/persistent/img_resized/164431928821...,BbR09i0h1Ow,None,None,None
1,1,/workspace/persistent/img_resized/164431930160...,BbR09i0h1Ow,None,None,None
2,2,/workspace/persistent/img_resized/164431931264...,BbR09i0h1Ow,None,None,None
3,3,/workspace/persistent/img_resized/164431932452...,BbR09i0h1Ow,None,None,None
4,4,/workspace/persistent/img_resized/184385171323...,BmWrV0xlpUk,None,None,None
...,...,...,...,...,...,...
1369450,1369450,/workspace/persistent/img_resized/182635175151...,BlYgT3YHdv-,None,None,None
1369451,1369451,/workspace/persistent/img_resized/180665928368...,BkSlNpBBE9v,None,None,None
1369452,1369452,/workspace/persistent/img_resized/180665931722...,BkSlNpBBE9v,None,None,None
1369453,1369453,/workspace/persistent/img_resized/182773686690...,BldbP8mhNZz,None,None,None


In [8]:
imgdf['image_hidden']=None
imgdf['image_attention']=None
imgdf['image_representation']=None

In [5]:
import os
import pandas as pd
import torch
import torchvision.transforms as transforms
from torchvision.models import inception_v3
from PIL import Image
from tqdm import tqdm

# Set device to CUDA if available, otherwise fall back to CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load InceptionV3 model
model = inception_v3(pretrained=True, aux_logits=True)
model = model.to(device)
model.eval()

# Create an empty list to store the feature vectors
feature_vectors = []

# Define the image loading and preprocessing function
transform = transforms.Compose([
    transforms.Resize((299, 299)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

# Iterate through the image files with tqdm
with tqdm(total=len(imgdf), desc="Processing Images") as progress_bar:
    for _, row in imgdf.iterrows():
        try:
            # Load and preprocess the image
            img = Image.open(row['pic'])
            img = transform(img)
            img = img.unsqueeze(0).to(device)  # Move image tensor to GPU

            # Get the feature vector
            with torch.no_grad():
                features = model(img)
                feature_vector = features.cpu()
            imgdf.at[row, 'image_hidden'] = feature_vector.tolist()

            # Update the progress bar
            progress_bar.update(1)
        except:
            pass

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Processing Images: 100%|█████████▉| 1368633/1369455 [6:58:23<00:15, 54.52it/s]  


In [26]:
pickle_file_path = 'tensor.pkl'
with open(pickle_file_path, 'wb') as file:
    pickle.dump(concatenated_tensor, file)

In [19]:
import pickle
pickle_file_path = 'tensor.pkl'
with open(pickle_file_path, 'rb') as file:
    tensor = pickle.load(file)

In [2]:
import torch
import torch.nn as nn
from tqdm import tqdm

# Define the self-attention module
class SelfAttention(nn.Module):
    def __init__(self, embedding_dim, num_heads):
        super(SelfAttention, self).__init__()
        self.attention = nn.MultiheadAttention(embed_dim=embedding_dim, num_heads=num_heads)

    def forward(self, x):
        # Apply self-attention
        output, _ = self.attention(x, x, x)

        return output.squeeze()

# Define the parameters
embedding_dim = 1000
num_heads = 4

# Create an instance of the self-attention module
self_attention = SelfAttention(embedding_dim, num_heads).cuda()

# Convert the given tensor to a PyTorch tensor and move to GPU
tensor = torch.tensor(tensor).cuda()

# Apply self-attention to all elements in the tensor
feature_vectors = []
with tqdm(total=len(imgdf), desc="Applying Attention") as progress_bar:
    for i,rows in imgdf.iterrows():
        # Apply self-attention
        output = self_attention(torch.tensor(imgdf['image_hidden'][i]).unsqueeze(0).to('cuda'))
        with torch.no_grad():
            output = output.cpu()
        imgdf['image_attention'][i]=output.tolist()
        progress_bar.update(1)


/tmp/ipykernel_391/3163351653.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tensor = torch.tensor(tensor).cuda()
Applying Attention: 100%|██████████| 1368633/1368633 [09:23<00:00, 2426.72it/s]


In [33]:
with tqdm(total=len(imgdf), desc="Calculating the Image Score") as progress_bar:
    for i,rows in imgdf.iterrows():
        xi=torch.tensor(imgdf['image_hidden'][i]).unsqueeze(0).to('cuda')*torch.tensor(imgdf['image_attention'][i]).unsqueeze(0).to('cuda')
        imgdf['image_representation'][i]=xi.tolist()
        progress_bar.update(1)

Calculating the Image Score: 100%|██████████| 1368633/1368633 [00:14<00:00, 92610.02it/s] 
